In [1]:
# Librerías
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Conexión del API con el programa
url = 'https://cobs.si/api/obs_list.api?des=C/2023 A3&format=json&from_date=&to_date=&exclude_faint=False&exclude_not_accurate=False'
response = requests.get(url) 

if response.status_code == 200:
    content = response.json()

In [3]:
# Creación de Dataset
cometa_df = pd.DataFrame(content['objects'])
cometa_df.sort_values('obs_date', ascending=False).info()

<class 'pandas.core.frame.DataFrame'>
Index: 1097 entries, 0 to 1096
Data columns (total 47 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   type                       1097 non-null   object 
 1   obs_date                   1097 non-null   object 
 2   comet                      1097 non-null   object 
 3   observer                   1097 non-null   object 
 4   location                   503 non-null    object 
 5   extinction                 50 non-null     object 
 6   obs_method                 1097 non-null   object 
 7   comet_visibility           1 non-null      object 
 8   magnitude                  1097 non-null   object 
 9   conditions                 14 non-null     object 
 10  ref_catalog                1097 non-null   object 
 11  instrument_aperture        1097 non-null   object 
 12  instrument_type            1097 non-null   object 
 13  instrument_focal_ratio     963 non-null    float64
 1

In [4]:
# Base de datos arrojada por la API
cometa_df.sample(5)

,type,obs_date,comet,observer,location,extinction,obs_method,comet_visibility,magnitude,conditions,...,magnitude_error,comparison_star_magnitude,pixel_size_x,pixel_size_y,pixel_size_unit,obs_comment,obs_sky_quality,obs_sky_quality_method,reference_star_names,date_added
447,V,2024-05-09 20:23:59,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Piotr', 'last_name': 'Guzik', ...",None,None,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,10.1,None,...,None,None,None,None,None,,None,NaN,,2024-05-09 20:42:38
616,V,2024-04-29 10:04:48,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Christopher', 'last_name': 'Wy...",None,&,"{'key': 'M', 'name': 'Modified-Out method', 's...",None,10.7,None,...,None,None,None,None,None,,None,NaN,,2024-04-29 11:07:45
50,V,2024-07-02 17:58:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Christian', 'last_name': 'Hard...","Capricorn, Namibia",None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,10.4,None,...,None,None,None,None,None,Der Komet stand im hellen Zodiakallicht !,21.2,3.0,,2024-07-18 15:29:57
685,V,2024-04-17 00:30:00,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Mike', 'last_name': 'Collins',...",Everton,None,"{'key': 'S', 'name': 'In-Out method', 'source'...",None,10.2,None,...,None,None,None,None,None,,19.1,2.0,"TYC 4971-0048, TYC 4971-0058",2024-04-17 13:48:22
908,C,2024-01-22 05:45:35,"{'type': 'C', 'name': 'C/2023 A3', 'fullname':...","{'first_name': 'Thomas', 'last_name': 'Lehmann...",Nerpio (remote),None,"{'key': 'Z', 'name': 'CCD Visual equivalent', ...",None,14.0,None,...,None,None,1.0,1.0,s,"CMOS/G, mlim=20.3",None,NaN,,2024-02-01 17:02:47


In [5]:
# Métodos de observación
cometa_df.obs_method.apply(lambda registro: f'{registro['key']}: {registro['name']}').value_counts()

obs_method
C: Unfiltered total CCD magnitude            323
Z: CCD Visual equivalent                     278
S: In-Out method                             185
M: Modified-Out method                       163
V: Johnson/Bessel/Kron/Cousins V with CCD     73
B: Simple Out-Out method                      19
k: Kron/Cousins R with CCD                    18
H: Kron/Cousins I with CCD                    17
D: Johnson/Bessel/Kron/Cousins B with CCD     10
c: Unfiltered nuclear CCD magnitude            8
d: Astrodon G filter                           2
E: Extrafocal-Extinction method                1
Name: count, dtype: int64

In [6]:
# Con valores nulos
filas,columnas = cometa_df.shape
print(f'Registros: {filas}\nVariables: {columnas + 1}')

Registros: 1097
Variables: 48


In [7]:
# Tratamiento de los datos de interés
cometa_df['obs_method_key'] = cometa_df.obs_method.apply(lambda registro: registro['key'])
cometa_df['obs_date'] = pd.to_datetime(cometa_df.obs_date)
cometa_df['magnitude'] = pd.to_numeric(cometa_df.magnitude)

In [8]:
# Info de nuevo data frame
curva_de_luz_df = cometa_df[['obs_method_key', 'obs_date', 'magnitude']]
curva_de_luz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   obs_method_key  1097 non-null   object        
 1   obs_date        1097 non-null   datetime64[ns]
 2   magnitude       1097 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.8+ KB


In [9]:
# Data Frame de la curva de luz
curva_de_luz_df.sample(5)

,obs_method_key,obs_date,magnitude
1058,Z,2023-04-22 00:00:00,17.2
2,M,2024-07-23 08:24:00,9.3
135,Z,2024-06-13 21:36:00,10.6
181,Z,2024-06-06 21:47:00,10.8
316,C,2024-05-20 21:21:36,10.6


In [10]:
# Gráfica del los datos
labels = {'obs_date':'Observation Date','magnitude':'Apparent total magnitude', 'obs_method_key' : 'Observation Method'}
fig = px.scatter(curva_de_luz_df, x='obs_date', y='magnitude', color='obs_method_key', template= 'plotly_dark', labels= labels, title='Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)')
fig.update_yaxes(autorange="reversed")
fig.show()

In [13]:
tiempo = curva_de_luz_df.obs_date
obs_method = curva_de_luz_df.obs_method_key.unique()


# Inicializar la figura con subplots
fig = make_subplots(rows=4, cols=3, subplot_titles= obs_method)

# Agregar trazas a los subplots
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[0]], mode="markers",name=obs_method[0]), row=1, col=1)
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[1]], mode="markers",name=obs_method[1]), row=1, col=2)
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[2]], mode="markers",name=obs_method[2]), row=1, col=3)

fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[3]], mode="markers",name=obs_method[3]), row=2, col=1)
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[4]], mode="markers",name=obs_method[4]), row=2, col=2)
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[5]], mode="markers",name=obs_method[5]), row=2, col=3)

fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[6]], mode="markers",name=obs_method[6]), row=3, col=1)
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[7]], mode="markers",name=obs_method[7]), row=3, col=2)
fig.add_trace(go.Scatter(x=tiempo, y=curva_de_luz_df.magnitude[curva_de_luz_df.obs_method_key == obs_method[8]], mode="markers",name=obs_method[8]), row=3, col=3)

# Actualizar el diseño de los subplots
fig.update_layout(title_text = 'Lightcurve of comet C/2023 A3 (Tsuchinshan-ATLAS)', template = 'plotly_dark')
fig.update_yaxes(autorange="reversed")

# Mostrar la figura
fig.show()